In [1]:
import pandas as pd
import numpy as np

import regex
import re

In [2]:
data = pd.read_json('salary_pred.json', encoding="UTF8")

In [3]:
data.head()

,_id,job_title,company,job_location,salary,summary,post_date
0,3d5487bc57a59ba4,nouveauData Scientist Big Data (F/H),AXA,Nanterre (92),,Gestion de projet data et data science. Animat...,Postedil y a 1 jour
1,de372ba90e161d50,nouveauData Analyst H/F,ACII,37000 Tours,35 000 € - 45 000 € par an,Une société participative et collaborative ave...,Postedil y a 6 jours
2,42b16956665ad440,Data Analyst,Daher,Tours (37),,Vous avez une appétence pour la data analyse e...,Postedil y a 30+ jours
3,f4faf36bbce2cda4,Data analyst campagne publicitaire Pinterest à...,Teleperformance Portugal,France,,"En tant qu’analyste publicitaire, votre rôle s...",Postedil y a 30+ jours
4,a17e6a7d7f9adb2e,Data Analyst F/H,AUDENSIEL ASSURANCES,Hybrid remote in Tours (37),,33 - 42 k€ brut annuel. Nous sommes à la reche...,Postedil y a 26 jours


In [4]:
#data description
data.describe()

,_id,job_title,company,job_location,salary,summary,post_date
count,5179,5179,5179,5179,5179,5179,5179
unique,5179,4367,2367,923,326,4747,60
top,3d5487bc57a59ba4,Business Developer H/F,Data Recrutement,Paris (75),,"Cette Talent Company rassemble Designers, Stra...",Postedil y a 30+ jours
freq,1,47,70,1391,4444,13,2600


In [5]:
#Data Informations
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5179 entries, 0 to 5178
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           5179 non-null   object
 1   job_title     5179 non-null   object
 2   company       5179 non-null   object
 3   job_location  5179 non-null   object
 4   salary        5179 non-null   object
 5   summary       5179 non-null   object
 6   post_date     5179 non-null   object
dtypes: object(7)
memory usage: 283.4+ KB


In [6]:
data.nunique()

_id             5179
job_title       4367
company         2367
job_location     923
salary           326
summary         4747
post_date         60
dtype: int64

In [7]:
#Checking missing values

data.isnull().sum()

_id             0
job_title       0
company         0
job_location    0
salary          0
summary         0
post_date       0
dtype: int64

### Texte processing

### Lowerization

In [8]:
data = data.applymap(lambda s: s.lower() if type(s) == str else s)

### Location text cleaning

In [9]:
data['job_location'] = data['job_location'].apply(lambda x: x.split('.')[0])

In [10]:
data['job_location'] = data['job_location'].apply(lambda x: x
                                    .replace('télétravail', '')
                                    .replace('temporaire', '')
                                    .replace('hybrid remote', '')
                                    .replace('in', '')
                                    .replace('+', '')
                                    .replace('lieu', ''))
data['job_location'].head()

0    nanterre (92)
1      37000 tours
2       tours (37)
3           france
4       tours (37)
Name: job_location, dtype: object

In [11]:
location_data = data['job_location'].replace(to_replace=r'.\d+.|[+]', value='', regex=True)

In [12]:
data['Location'] = location_data.apply(lambda x: x.split('-')[0]).str.capitalize()
data['Location'].head()

0    Nanterre 
1        Tours
2       Tours 
3       France
4       tours 
Name: Location, dtype: object

### Salary processing: Minimum, maximum et mean

### Salary features: salary pay type

par heure, par jour, par semaine, par mois, par an

In [13]:
#Creating columns for each salay pay type
data['per_hour'] = data['salary'].apply(lambda x: 1 if 'par heure' in x else 0 )

In [14]:
data['per_day'] = data['salary'].apply(lambda x: 1 if 'par jour' in x else 0 )

In [15]:
data['per_week'] = data['salary'].apply(lambda x: 1 if 'par semaine' in x else 0 )

In [16]:
data['per_month'] = data['salary'].apply(lambda x: 1 if 'par mois' in x else 0 )

In [17]:
data['per_year'] = data['salary'].apply(lambda x: 1 if 'par an' in x else 0 )

### salary text cleaning

In [18]:
#removing (,) and replacing it by a (.)
first_clean = data['salary'].str.replace(',', '.')

In [19]:
#removing all characters[a-z] except figures[0-9], regex
clean_salary = first_clean.replace(to_replace=r'[a-z]|[€]|[ ]', value='', regex=True)

### Salary data analysis

#### Calculating the minimum salary

In [20]:
#spliting the range salary 1st value[0], as min_sal = minimum salary, str

min_salary = clean_salary.apply(lambda x: x.split('-')[0])

In [21]:
#converting to numeric values, float type

minimum = pd.to_numeric(min_salary, errors='coerce').astype(float)

### Minimum Annual salary

In [22]:
#Annual minimum salary

m1 = data['salary'].str.contains('par heure')
m2 = data['salary'].str.contains('par jour')
m3 = data['salary'].str.contains('par semaine')
m4 = data['salary'].str.contains('par mois')

data['minimum_annual_salary'] = np.select([m1, m2, m3, m4],
                                  [minimum*1825, #converting hourly to annual
                                   minimum*260,   #converting daily to annual                               
                                   minimum*52,    #weeklyly to annual
                                   minimum*12],   #monthly to annual
                                   default=minimum) #default value
#data['minimum_annual_salary'].describe().round()

### Minimum Monthly salary

In [23]:
#Monthly minimum salary
w1 = data['salary'].str.contains('par heure')
w2 = data['salary'].str.contains('par jour')
w3 = data['salary'].str.contains('par semaine')
w4 = data['salary'].str.contains('par an')

data['minimum_monthly_salary'] = np.select([w1, w2, w3, w4],
                                  [minimum*152, #converting hourly to monthly
                                   minimum*22,   #converting daily to monthly                               
                                   minimum*4,    #weeklyly to monthly
                                   minimum/12],   #annual to monthly
                                   default=minimum) #default value
#data['minimum_monthly_salary'].describe().round()

### Calculating the maximum salary

In [24]:
#spliting the range salary 2nd value[1], as max_sal = maximum salary, str

max_salary = clean_salary.apply(lambda x: x.split('-')[1] if len(x.split('-'))>1 else x)

In [25]:
#numeric value
maximum = pd.to_numeric(max_salary, errors='coerce').astype(float)

### Annual Maximum salary

In [26]:
#Annual maximum salary

m1 = data['salary'].str.contains('par heure')
m2 = data['salary'].str.contains('par jour')
m3 = data['salary'].str.contains('par semaine')
m4 = data['salary'].str.contains('par mois')

data['maximum_annual_salary'] = np.select([m1, m2, m3, m4],
                                  [maximum*1825, #converting hourly to annual
                                   maximum*260,   #converting daily to annual                               
                                   maximum*52,    #weeklyly to annual
                                   maximum*12],   #monthly to annual
                                   default=maximum) #default value
#data['maximum_monthly_salary'].describe().round()

In [27]:
#Monthly salary
w1 = data['salary'].str.contains('par heure')
w2 = data['salary'].str.contains('par jour')
w3 = data['salary'].str.contains('par semaine')
w4 = data['salary'].str.contains('par an')

data['maximum_monthly_salary'] = np.select([w1, w2, w3, w4],
                                  [maximum*152, #converting hourly to monthly
                                   maximum*22,   #converting daily to monthly                               
                                   maximum*4,    #weeklyly to monthly
                                   maximum/12],   #annual to monthly
                                   default=maximum) #default value
#data['maximum_monthly_salary'].describe().round()

### Average Salary

#### Average Annual Salary

In [28]:
# Annual average salary; (min + max)/2

data['annual_average_salary'] = (data['minimum_annual_salary'] + data['maximum_annual_salary'])/2

#### Average Monthly salary

In [32]:
data['monthly_average_salary'] = (data['minimum_monthly_salary'] + data['maximum_monthly_salary'])/2

Reference: 

1. <a href="https://www.geeksforgeeks.org/get-all-rows-in-a-pandas-dataframe-containing-given-substring/">Get all rows in a Pandas DataFrame containing</a>

2. <a href="https://nycdatascience.com/blog/student-works/job-salary-prediction-with-python/">Job Salary Prediction with Python</a>

3. <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html">pandas.Series.replace</a>

In [33]:
data.head()

,_id,job_title,company,job_location,salary,summary,post_date,Location,per_hour,per_day,per_week,per_month,per_year,minimum_annual_salary,minimum_monthly_salary,maximum_annual_salary,maximum_monthly_salary,annual_average_salary,monthly_average_salary
0,3d5487bc57a59ba4,nouveaudata scientist big data (f/h),axa,nanterre (92),,gestion de projet data et data science. animat...,postedil y a 1 jour,Nanterre,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,de372ba90e161d50,nouveaudata analyst h/f,acii,37000 tours,35 000 € - 45 000 € par an,une société participative et collaborative ave...,postedil y a 6 jours,Tours,0,0,0,0,1,35000.0,2916.666667,45000.0,3750.0,3333.333333,3333.333333
2,42b16956665ad440,data analyst,daher,tours (37),,vous avez une appétence pour la data analyse e...,postedil y a 30+ jours,Tours,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,f4faf36bbce2cda4,data analyst campagne publicitaire pinterest à...,teleperformance portugal,france,,"en tant qu’analyste publicitaire, votre rôle s...",postedil y a 30+ jours,France,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,a17e6a7d7f9adb2e,data analyst f/h,audensiel assurances,tours (37),,33 - 42 k€ brut annuel. nous sommes à la reche...,postedil y a 26 jours,tours,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
